In [6]:
df = pd.read_csv("features.csv", sep=";")
df = df.drop('semanticobjscore', axis=1) 
df = df.drop('semanticsubjscore', axis=1) 
df = df.drop('URL', axis=1) 

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


X = df.drop(['Label', 'TextID'], axis=1)  
y = df['Label']

# Convertir les labels en nombres
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Classes:", le.classes_)  # Pour voir la correspondance

# Refaire la séparation train/test/val avec les labels encodés
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_encoded,
    test_size=0.2,
    random_state=3,
    stratify=y_encoded
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,
    random_state=3,
    stratify=y_temp
)

Classes: ['objective' 'subjective']


In [13]:
# NEURAL NETWORK STEP BY STEP

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val) 
X_test_scaled = scaler.transform


model = MLPClassifier(
    hidden_layer_sizes=(100, 50),  
    max_iter=1000,                  
    random_state=3,
    verbose=1                       
)

# ===== ÉTAPE 3 : Entraîner le modèle =====
# Utilise les données d'entraînement normalisées
print("Entraînement du modèle...")
model.fit(X_train, y_train)
print("Entraînement terminé!\n")

# ===== ÉTAPE 4 : Faire des prédictions =====
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

# ===== ÉTAPE 5 : Évaluer les performances =====
print("=" * 50)
print("RÉSULTATS DU NEURAL NETWORK")
print("=" * 50)

val_acc = accuracy_score(y_val, y_pred_val) 
val_precision = precision_score(y_val, y_pred_val)
val_recall = recall_score(y_val, y_pred_val)
val_f1 = f1_score(y_val, y_pred_val)

train_acc = accuracy_score(y_train, y_pred_train)
train_precision = precision_score(y_train, y_pred_train)
train_recall = recall_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train)

print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Train Precision: {train_precision:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Train Recall: {train_recall:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Train F1: {train_f1:.4f}")
print(f"Validation F1-score: {val_f1:.4f}")
print("\n")
# ===== ÉTAPE 6 (OPTIONNEL) : Infos sur le modèle =====
print("\n" + "=" * 50)
print("INFOS SUR LE MODÈLE")
print("=" * 50)
print(f"Nombre de couches : {len(model.hidden_layer_sizes) + 1}")  # +1 pour output
print(f"Nombre de paramètres (poids) : {model.n_layers_}")
print(f"Iterations effectuées : {model.n_iter_}")
print(f"Loss (erreur finale) : {model.loss_:.4f}")

Entraînement du modèle...
Iteration 1, loss = 7.94522992
Iteration 2, loss = 5.71048769
Iteration 3, loss = 3.63920185
Iteration 4, loss = 2.50007357
Iteration 5, loss = 2.53778353
Iteration 6, loss = 1.81859237
Iteration 7, loss = 1.41921459
Iteration 8, loss = 1.26513283
Iteration 9, loss = 1.13438602
Iteration 10, loss = 0.96698646
Iteration 11, loss = 0.84822269
Iteration 12, loss = 0.86998834
Iteration 13, loss = 0.73834944
Iteration 14, loss = 0.74539767
Iteration 15, loss = 0.72717248
Iteration 16, loss = 0.69485968
Iteration 17, loss = 0.67831169
Iteration 18, loss = 0.67863872
Iteration 19, loss = 0.64883305
Iteration 20, loss = 0.62376573
Iteration 21, loss = 0.62164284
Iteration 22, loss = 0.60876087
Iteration 23, loss = 0.57811315
Iteration 24, loss = 0.56658762
Iteration 25, loss = 0.55256591
Iteration 26, loss = 0.53608111
Iteration 27, loss = 0.51722324
Iteration 28, loss = 0.51216874
Iteration 29, loss = 0.50623162
Iteration 30, loss = 0.51958057
Iteration 31, loss = 0.

La fonction MLPClassifier prend par défaut ReLu comme fonction d'activation et la fonction sigmoïde pour la couche de sortie (car on est dans un cas binaire). L'algorithme par défaut est l'algorithme Adam (avec taux d'apprentissage 0,001 et taux de régularisation 0,0001)

LÀ ON VA UTILISER RANDOMSEARCH POUR CHOISIR LES HYPERPARAMÈTRES AU MIEUX 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from scipy.stats import loguniform, uniform, randint # Pour définir des distributions
from sklearn.metrics import f1_score # La métrique sur laquelle arbitrer

# Définition du modèle de base (sans hyperparamètres spécifiques)
# C'est l'objet qui sera cloné et entraîné par la recherche aléatoire
model_base = MLPClassifier(max_iter=2000, random_state=42)



# Nous définissons l'espace de recherche (le "param_dist")
param_dist = {
    # 1. Architecture (hidden_layer_sizes) : C'est une liste de tuples à choisir
    'hidden_layer_sizes': [(50,), (100,), (50, 25), (100, 50), (128, 64)], 
    
    # 2. Régularisation (alpha) : Loguniform est idéal pour les paramètres de régularisation
    'alpha': loguniform(1e-5, 1e-2), # Exemple: choisir une valeur entre 0.00001 et 0.01
    
    # 3. Optimisation (learning_rate_init) : Loguniform est aussi bon pour le taux d'apprentissage
    'learning_rate_init': loguniform(1e-4, 1e-2), # Exemple: choisir une valeur entre 0.0001 et 0.01
    
    # 4. Fonction d'activation : Une liste de choix discrets
    'activation': ['tanh', 'relu'],
    
    # 5. Type de solveur : Une liste de choix discrets
    'solver': ['adam', 'sgd'] 
}


# Instanciation de RandomizedSearchCV
# n_iter=50 : Le nombre total d'itérations aléatoires que nous voulons tester (compromis temps/précision)
# scoring='f1' : La métrique sur laquelle nous arbitrons (maximiser le F1-score)
# cv=5 : Utilise la validation croisée K-Fold avec 5 plis (plus fiable qu'un simple split Train/Val)
# verbose=2 : Affiche le progrès détaillé
# n_jobs=-1 : Utilise tous les cœurs du processeur pour accélérer le calcul
random_search = RandomizedSearchCV(
    estimator=model_base,
    param_distributions=param_dist,
    n_iter=50, 
    scoring='f1', 
    cv=5, 
    random_state=42, 
    verbose=2,
    n_jobs=-1 
)

# X_train et y_train sont les données utilisées pour la recherche d'hyperparamètres.
# L'objet random_search effectue l'entraînement et la validation croisée en interne.
print("Début de la recherche aléatoire...")
random_search.fit(X_train, y_train) 
print("Recherche aléatoire terminée.")


# Récupération du meilleur jeu d'hyperparamètres (celui qui a maximisé le F1-score CV)
best_params = random_search.best_params_

# Récupération du modèle qui a obtenu le meilleur score de validation croisée
best_model = random_search.best_estimator_

print("\n---------------------------------------------")
print("MEILLEURS HYPERPARAMÈTRES TROUVÉS (JUSTIFICATION) :")
print(best_params)
print(f"Meilleur score de validation croisée (F1-score) : {random_search.best_score_:.4f}")
print("---------------------------------------------")

# Évaluation finale sur l'ensemble de TEST
y_pred_test = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred_test)

print(f"Performance finale sur la base de TEST : F1-score = {test_f1:.4f}")